In [73]:
library(dplyr)
library(class)
library(caret)
library(glm2)
library(glmnet)
library(pROC)
library(randomForest)
library(ggplot2)
library(purrr)
library(broom)
library(stringr)
library(GGally)
library(broom)
install.packages("stargazer")
library(stargazer)


The downloaded binary packages are in
	/var/folders/x1/7zk8m6y57m1d830596wr0ngw0000gn/T//RtmpP7yjcw/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




In [60]:
df <- read.csv("train.csv")

In [61]:
long_colnames <- names(df)
names(df) <- c(
  "age",
  "gender",
  "transgender",
  "race_ethnicity",
  "marital_status",
  "CRF",
  "phys_activity_past_week",
  "sweet_drinks_past_month",
  "fruit_past_month",
  "veg_past_month",
  "tobacco_past_30days",
  "cigarettes_per_day",
  "binge_drinking_past_30days",
  "bmi",
  "drug_use",
  "psych_distress_score",
  "disability_status",
  "pregnant",
  "medicare_coverage",
  "medical_coverage",
  "insured",
  "insurance_type_under65",
  "insurance_type_over65",
  "zipcode_type",
  "doctor_visits_past_year",
  "insurance_months_past_year",
  "poverty_level_fpl"
)

In [62]:
# Convert CRF 'yes'/'no' to 1/0
df$CRF <- ifelse(df$CRF == "YES", 1, 0)

# combine 'type of insurance' columns (reported in separate columns for <65 and >=65 yrs) and delete original cols
df$current_insurance_type <- ifelse(
    df$insurance_type_under65 == 'SKIPPED - AGE >= 65',
    df$insurance_type_over65, 
    df$insurance_type_under65)
df <- df %>% select(-insurance_type_under65, -insurance_type_over65)

# remove continuous outcome variable from df
df <- df %>% select(-doctor_visits_past_year)

# Convert all categorical predictor variables to factors
df[] <- lapply(df, function(x) if (is.character(x)) factor(x) else x)
#str(df) # Check the structure of the dataframe

# Convert outcome to factor
df$CRF <- factor(df$CRF, levels = c(0,1))

# Specify the binary outcome variable and predictor variables
outcome <- "CRF"
# exclude outcome variable from predictors
predictors <- names(df)[names(df) != "CRF"]

In [63]:
# total row count in train set
n <- nrow(df)

# Calculate proportion of positives in data
    # YES = has been diagnosed with chronic risk factor

RF <- sum(df$CRF == 1, na.rm = TRUE)
NoRF <- sum(df$CRF == 0, na.rm = TRUE)
propRF <- round(RF / n, 3)
propNoRF <- 1 - propRF # no missing data for this variable

print(paste('# with risk factor Dx: ', RF))
print(paste('# without risk factor Dx: ', NoRF))
print(paste('% with risk factor Dx: ', propRF))
print(paste('% without risk factor Dx: ', propNoRF))

# Confusion matrix and accuracy for classifier that always predicts 1

# Convert always-positive predictions to factor
predictions <- rep(1, n)
predictions <- factor(predictions, levels = c(1,0))

cat("Confusion Matrix:\n")
print(confusionMatrix(predictions, df$CRF))

# calculate accuracy
TP <- RF
TN <- 0
acc <- (TP + TN) / n
print(paste('Baseline accuracy: ', acc))

[1] "# with risk factor Dx:  1846"
[1] "# without risk factor Dx:  1124"
[1] "% with risk factor Dx:  0.622"
[1] "% without risk factor Dx:  0.378"
Confusion Matrix:


Warning message in confusionMatrix.default(predictions, df$CRF):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0    0    0
         1 1124 1846
                                         
               Accuracy : 0.6215         
                 95% CI : (0.6038, 0.639)
    No Information Rate : 0.6215         
    P-Value [Acc > NIR] : 0.5082         
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 0.0000         
            Specificity : 1.0000         
         Pos Pred Value :    NaN         
         Neg Pred Value : 0.6215         
             Prevalence : 0.3785         
         Detection Rate : 0.0000         
   Detection Prevalence : 0.0000         
      Balanced Accuracy : 0.5000         
                                         
       'Positive' Class : 0              
                                         
[1

In [64]:
set.seed(123)  # for reproducibility
k <- 5  # number of folds
folds <- createFolds(df$CRF, k = k, list = TRUE, returnTrain = FALSE)


## Strategy 1: Logistic Regression with all covariates
# Create the formula
formula <- as.formula(paste(outcome, "~", paste(predictors, collapse = "+")))

# 5-fold CV
k <- 5

# Initialize vectors to store performance metrics
accuracies <- numeric(k)
sensitivities <- numeric(k)
specificities <- numeric(k)

# Perform k-fold cross-validation
for (i in 1:k) {
  # Split data into training and testing sets
  test_indices <- folds[[i]]
  train_data <- df[-test_indices, ]
  test_data <- df[test_indices, ]
  
  # Fit logistic regression model
  model <- glm(formula = formula, data = train_data, family = "binomial")
  
  # Make predictions on test data
  predictions <- predict(model, newdata = test_data, type = "response")
  predicted_classes <- ifelse(predictions > 0.5, 1, 0) # default threshold = 0.5
  
  # Calculate performance metrics
  confusion_matrix <- table(test_data$CRF, predicted_classes)
  accuracies[i] <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
  sensitivities[i] <- confusion_matrix[2, 2] / sum(confusion_matrix[2, ])
  specificities[i] <- confusion_matrix[1, 1] / sum(confusion_matrix[1, ])
  
}

# Calculate average performance metrics
mean_accuracy <- mean(accuracies)
mean_sensitivity <- mean(sensitivities)
mean_specificity <- mean(specificities)

# Print results
cat("Logistic regression performance:", "\n")
cat("Mean Accuracy:", mean_accuracy, "\n")
cat("Mean Sensitivity:", mean_sensitivity, "\n")
cat("Mean Specificity:", mean_specificity, "\n")

Logistic regression performance: 
Mean Accuracy: 0.7131032 
Mean Sensitivity: 0.8336893 
Mean Specificity: 0.5150437 


In [78]:
# Fit the logistic regression model on the entire train set
model <- glm(formula = formula, data = df, family = binomial())

In [51]:
test_df <- read.csv('test.csv')

long_colnames <- names(test_df)
names(test_df) <- c(
  "age",
  "gender",
  "transgender",
  "race_ethnicity",
  "marital_status",
  "CRF",
  "phys_activity_past_week",
  "sweet_drinks_past_month",
  "fruit_past_month",
  "veg_past_month",
  "tobacco_past_30days",
  "cigarettes_per_day",
  "binge_drinking_past_30days",
  "bmi",
  "drug_use",
  "psych_distress_score",
  "disability_status",
  "pregnant",
  "medicare_coverage",
  "medical_coverage",
  "insured",
  "insurance_type_under65",
  "insurance_type_over65",
  "zipcode_type",
  "doctor_visits_past_year",
  "insurance_months_past_year",
  "poverty_level_fpl"
)

# Convert CRF 'yes'/'no' to 1/0
test_df$CRF <- ifelse(test_df$CRF == "YES", 0, 1)

# combine 'type of insurance' columns (reported in separate columns for <65 and >=65 yrs) and delete original cols
test_df$current_insurance_type <- ifelse(
    test_df$insurance_type_under65 == 'SKIPPED - AGE >= 65',
    test_df$insurance_type_over65, 
    test_df$insurance_type_under65)
test_df <- test_df %>% select(-insurance_type_under65, -insurance_type_over65)

# remove continuous outcome variable from df
test_df <- test_df %>% select(-doctor_visits_past_year)

# Convert all categorical predictor variables to factors
test_df[] <- lapply(test_df, function(x) if (is.character(x)) factor(x) else x)

# Convert outcome to factor
test_df$CRF <- factor(test_df$CRF, levels = c(0,1))

In [55]:
predictions <- predict(model, newdata = test_df, type = "response")
predicted_classes <- ifelse(predictions > 0.5, 0, 1) # default threshold = 0.5

  # Calculate performance metrics
confusion_matrix <- table(test_df$CRF, predicted_classes)
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat("Logistic regression performance on test set:", "\n")
cat("Accuracy:", accuracy, "\n")

Logistic regression performance on test set: 
Accuracy: 0.722372 


In [89]:
# Extract significant coefficients (p-value < 0.05)
# Summary of the model - chosen model fit on train set
summary_output <- summary(model)

significant_coefficients <- summary_output$coefficients[
  summary_output$coefficients[, "Pr(>|z|)"] < 0.05, 
]
print("Significant coefficients for chosen model on train set:")
print(significant_coefficients)

[1] "Significant coefficients for chosen model on train set:"
                                                              Estimate
age40-44 YEARS                                              0.71078159
age45-49 YEARS                                              0.94148278
age50-54 YEARS                                              1.12430148
age55-59 YEARS                                              1.22690968
age60-64 YEARS                                              1.80765664
age65-69 YEARS                                              2.67741648
age70-74 YEARS                                              2.97747753
age75-79 YEARS                                              3.41796260
age80-84 YEARS                                              2.83654648
age85+ YEARS                                                3.25380898
marital_statusNEVER MARRIED                                -0.38378758
bmi23.0 - 27.49 (OVERWEIGHT OR OBESE, WITH INCREASED RISK)  0.79116643
bmi27.5 OR HIGH

In [94]:
# Get p-values of coefficients
p_values <- coef(summary_output)[, "Pr(>|z|)"]

# Apply Benjamini-Hochberg procedure
p_adjusted <- p.adjust(p_values, method = "BH")

# Combine results
results <- data.frame(
  Adjusted_P_Value = p_adjusted
)

# Filter for adjusted p-values smaller than 0.05
significant_results <- results %>%
  filter(Adjusted_P_Value < 0.05)

# Print the significant adjusted p-values
print(significant_results)

                                                        Adjusted_P_Value
age40-44 YEARS                                              2.054724e-02
age45-49 YEARS                                              2.335502e-03
age50-54 YEARS                                              1.558342e-04
age55-59 YEARS                                              3.939614e-05
age60-64 YEARS                                              1.676284e-10
bmi27.5 OR HIGHER (OVERWEIGHT OR OBESE, WITH HIGH RISK)     8.845272e-05
psych_distress_score                                        9.086464e-03
disability_statusNOT DISABLED                               2.790137e-08
medicare_coverageYES                                        2.651323e-02


In [88]:
reasonable_predictor_columns = list(
  "age",
  "gender",
  "transgender",
  "race_ethnicity",
  "marital_status",
  "phys_activity_past_week",
  "sweet_drinks_past_month",
  "fruit_past_month",
  "veg_past_month",
  "tobacco_past_30days",
  "cigarettes_per_day",
  "binge_drinking_past_30days",
  "bmi",
  "drug_use",
  "psych_distress_score",
  "disability_status",
  "insured",
  "zipcode_type"
)
reasonable_predictors <- names(df)[names(df) %in% reasonable_predictor_columns]

formula_with_subset_of_coeff = as.formula(paste(outcome, "~ ", paste(reasonable_predictors, collapse = "+")))

subset_of_coeff_model = glm(formula = formula_with_subset_of_coeff, data = df, family = binomial())

summary_output <- summary(subset_of_coeff_model)
significant_coefficients <- summary_output$coefficients[
  summary_output$coefficients[, "Pr(>|z|)"] < 0.05, 
]
print("Significant coefficients for model with 'reasonable' subset of predictors:")
print(significant_coefficients)

[1] "Significant coefficients for model with 'reasonable' subset of predictors:"
                                                              Estimate
age40-44 YEARS                                              0.75277342
age45-49 YEARS                                              1.10862539
age50-54 YEARS                                              1.34514979
age55-59 YEARS                                              1.47025611
age60-64 YEARS                                              2.02465982
age65-69 YEARS                                              2.10775936
age70-74 YEARS                                              2.36942216
age75-79 YEARS                                              2.78978811
age80-84 YEARS                                              2.22913086
age85+ YEARS                                                2.68566189
bmi23.0 - 27.49 (OVERWEIGHT OR OBESE, WITH INCREASED RISK)  0.76764557
bmi27.5 OR HIGHER (OVERWEIGHT OR OBESE, WITH HIGH RISK)     1.50215

In [87]:
model_fit_on_test_data <- glm(formula = formula, data = test_df, family = binomial())

summary_output <- summary(model_fit_on_test_data)
significant_coefficients <- summary_output$coefficients[
  summary_output$coefficients[, "Pr(>|z|)"] < 0.05, 
]
print("Significant coefficients for chosen model on test set:")
print(significant_coefficients)

predictions <- predict(model_fit_on_test_data, newdata = test_df, type = "response")
predicted_classes <- ifelse(predictions > 0.5, 1, 0) # default threshold = 0.5

  # Calculate performance metrics
confusion_matrix <- table(test_df$CRF, predicted_classes)
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat("Accuracy:", accuracy, "\n")

[1] "Significant coefficients for chosen model on test set:"
                              Estimate Std. Error   z value    Pr(>|z|)
age60-64 YEARS             -1.06772701 0.51859710 -2.058876 0.039506141
genderMALE                  0.51825167 0.24255718  2.136616 0.032629198
psych_distress_score       -0.04584952 0.02128328 -2.154251 0.031220508
pregnantNO                  0.92822265 0.39796054  2.332449 0.019677085
insuredYES                  2.72708265 1.05203646  2.592194 0.009536596
insurance_months_past_year -0.23053994 0.08633031 -2.670440 0.007575184
Accuracy: 0.7654987 
